<a href="https://colab.research.google.com/github/Rtniewi/kcwiertniewicz-IDS/blob/main/GenAI/HW5/LSTM_1__Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

//***Katrina Cwiertniewicz
//*** CSC 330
//11/22/2024
//Assignment 5: Text Generation Using LSTM on Project Gutenberg Training Data
####The purpose of this assignment is to develop an LSTM model that generates text. The goal is to produce coherent and stylistically relevant text based on prompts.

In [1]:
import numpy as np
import json
import re
import string

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, losses

## 0. Parameters <a name="parameters"></a>

In [2]:
VOCAB_SIZE = 20049
MAX_LEN = 500
EMBEDDING_DIM = 100
N_UNITS = 128
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 25

## 1. Load the data <a name="load"></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%pwd

'/content'

In [5]:
import requests
import os


# List of files for additional texts (e.g., different Edgar Allan Poe Works)
file_paths = [
  os.path.join('/content/drive/MyDrive/text/The_Tell_Tale_Heart.txt'),                        # The Tell Tale Heart
  os.path.join('/content/drive/MyDrive/text/The_Cask_of_Amontillado.txt'),                    # The Cask of Amontillado
  os.path.join('/content/drive/MyDrive/text/The_Raven.txt'),                                  # The Raven
  os.path.join('/content/drive/MyDrive/text/The_Masque.txt'),                                 # The Masque of the Red Death
  os.path.join('/content/drive/MyDrive/text/Annabel_Lee.txt'),                                # Annabel Lee
  os.path.join('/content/drive/MyDrive/text/Lenore.txt'),                                     # Lenore
  os.path.join('/content/drive/MyDrive/text/The_Bells.txt'),                                  # The Bells
  os.path.join('/content/drive/MyDrive/text/The_Black_Cat.txt'),                              # The Black Cat
  os.path.join('/content/drive/MyDrive/text/The_Fall_of_the_House_of_Usher.txt'),             # The Fall of the House of Usher
  os.path.join('/content/drive/MyDrive/text/The_Oval_Portrait.txt'),                          # The Oval Portrait
  os.path.join('/content/drive/MyDrive/text/The_Pit_and_the_Pendulum.txt'),                   # The Pit and the Pendulum
  os.path.join('/content/drive/MyDrive/text/The_Premature_Burial.txt'),                       # The Premature Burial
  os.path.join('/content/drive/MyDrive/text/The_Narrative_of_Arthur_Gordon.txt'),             # The Narrative of Arthur Gordon Pym of Nantucket
  os.path.join('/content/drive/MyDrive/text/Al_Aaraaf.txt'),                                  # Al Aaraaf
  os.path.join('/content/drive/MyDrive/text/Alone.txt'),                                      # Alone
  os.path.join('/content/drive/MyDrive/text/Eureka.txt'),                                     # Eureka
  os.path.join('/content/drive/MyDrive/text/The_Facts_in_the_Case_of_M._Valdemar.txt'),       # The Facts in the Case of M.Valdemar
  os.path.join('/content/drive/MyDrive/text/A_Descent_into_the_Maelstrom.txt'),               # A Descent into the Maelstrom
  os.path.join('/content/drive/MyDrive/text/William_Wilson.txt'),                             # William Wilson
  os.path.join('/content/drive/MyDrive/text/Berenice.txt'),                                   # Berenice
  os.path.join('/content/drive/MyDrive/text/The_Gold_Bug.txt'),                               # The Gold Bug
  os.path.join('/content/drive/MyDrive/text/The_Murders_of_Rue_Morgue.txt'),                  # The Murders in the Rue Morgue
  os.path.join('/content/drive/MyDrive/text/The_Mystery_of_Marie_Roget.txt'),                 # The Mystery of Marie Roget
  os.path.join('/content/drive/MyDrive/text/The_Purloined_Letter.txt'),                       # The Purloined Letter
  os.path.join('/content/drive/MyDrive/text/Von_Kempelen_and _his_Discovery.txt'),            # Von Kempelen and His Discovery
  os.path.join('/content/drive/MyDrive/text/Island_of_the_Fay.txt'),                          # Island of the Fay
  os.path.join('/content/drive/MyDrive/text/Mesmeric_Revelation.txt'),                        # Mesemeric Revelation
  os.path.join('/content/drive/MyDrive/text/Silence_A_Fable.txt'),                            # Silence a Fable
  os.path.join('/content/drive/MyDrive/text/MS._Found_in_a_Bottle.txt'),                      # MS. Found in a Bottle
  os.path.join('/content/drive/MyDrive/text/The_Thousand_and_Second_Tale_of_Scherezade.txt'), # The Thousand and Second Tale of Scherezade
  os.path.join('/content/drive/MyDrive/text/The_Unparalleled_Adventure.txt'),                 # The Unparalleled Adventure of One Hans Pfaall
  os.path.join('/content/drive/MyDrive/text/The_Assignation.txt'),                            # The Assignation
  os.path.join('/content/drive/MyDrive/text/The_Imp.txt'),                                    # The Imp of the Perverse
  os.path.join('/content/drive/MyDrive/text/The_Domain_of_Arnheim.txt'),                      # The Domain of Arnheim
  os.path.join('/content/drive/MyDrive/text/The_Assignation.txt'),                            # Landor's Cottage
  os.path.join('/content/drive/MyDrive/text/Morella.txt'),                                    # Morella
  os.path.join('/content/drive/MyDrive/text/Ligeia.txt'),                                     # Ligeia
  os.path.join('/content/drive/MyDrive/text/King_Pest.txt'),                                  # King Pest
  os.path.join('/content/drive/MyDrive/text/A_Tale_of_the_Ragged_Mountains.txt'),             # A Tale of the Ragged Mountains
  os.path.join('/content/drive/MyDrive/text/The_Spectacles.txt'),                             # The Spectacles
  os.path.join('/content/drive/MyDrive/text/Philosophy_of_Furniture.txt'),                    # The Philosophy of Furniture
  os.path.join('/content/drive/MyDrive/text/The_Devil_in_Belfry.txt'),                        # The Devil in the Belfry
  os.path.join('/content/drive/MyDrive/text/Bon_Bon.txt'),                                    # Bon-Bon
  os.path.join('/content/drive/MyDrive/text/Some_Words_with_a_Mummy.txt')                     # Some Words with a Mummy


]

# Initialize an empty string to hold all text
all_text = ""

# Download each text file and append to all_text
for file_path in file_paths:
  with open(file_path, 'r') as file:
    content = file.read()
    text = content
    all_text += text + "\n\n"  # Separate texts by newlines

# Save combined text to a single file
  with open('/content/combined_poe.txt', "w", encoding="utf-8") as file:
    file.write(all_text)


In [6]:
# Count the words of text
with open('/content/combined_poe.txt', "r", encoding="utf-8") as file:
  file_content = file.read()
  words = file_content.split()
  n_words = len(words)
print(f"{n_words} words loaded")

326500 words loaded


In [7]:
# Example Sentence of First Ten Words
example_sentence = words[:10]
print(f"Example Sentence: {example_sentence}")

Example Sentence: ['True!—nervous—very,', 'very', 'dreadfully', 'nervous', 'I', 'had', 'been', 'and', 'am;', 'but']


## 2. Tokenise the data

In [8]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


with open("combined_poe.txt", "r", encoding="utf-8") as file:
    text_data = [pad_punctuation(line) for line in file]

In [9]:
example_date = text_data[30]
print(example_date)

Ha ! —would a madman have been so wise as this ? And then , when my



In [10]:
# Convert to a Tensorflow Dataset
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [11]:
# Create a vectorisation layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [12]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

# Length of Vocabulary
print(f"Length of Vocabulary: {len(vocab)}")

Length of Vocabulary: 20049


In [13]:
# Display some token:word mappings
for i, word in enumerate(vocab[10:30]):
    print(f"{i}: {word}")

0: -
1: i
2: that
3: _
4: it
5: was
6: as
7: with
8: which
9: at
10: my
11: is
12: had
13: ;
14: we
15: this
16: for
17: by
18: not
19: be


In [14]:
# Display the same example converted to ints
example_tokenised = vectorize_layer(text_data)
print(example_tokenised.numpy())

[[  326    47 11929 ...     0     0     0]
 [   33   329    74 ...     0     0     0]
 [13630 18134 10712 ...     0     0     0]
 ...
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]]


## 3. Create the Training Set

In [15]:
# Create the training set of text and the same text shifted by one word
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

## 4. Build the LSTM <a name="build"></a>

In [17]:
inputs = layers.Input(shape=(None,), dtype="int32")
x = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x = layers.LSTM(N_UNITS, return_sequences=True)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
lstm = models.Model(inputs, outputs)
lstm.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_1 (Embedding)              │ (None, None, 100)           │       2,004,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, None, 128)           │         117,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, None, 20049)         │       2,586,321 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,708,469 (17.96 MB)

 Trainable params: 4,708,469 (17.96 MB)

 Non-trainable params: 0 (0.00 B)

## 5. Train the LSTM <a name="train"></a>

In [18]:
loss_fn = losses.SparseCategoricalCrossentropy()
lstm.compile("adam", loss_fn)

In [19]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }  # <1>

    def sample_from(self, probs, temperature):  # <2>
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]  # <3>
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0: # <4>
            x = np.array([start_tokens])
            y = self.model.predict(x, verbose=0)  # <5>
            sample_token, probs = self.sample_from(y[0][-1], temperature)  # <6>
            info.append({"prompt": start_prompt, "word_probs": probs})
            start_tokens.append(sample_token)  # <7>
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("Text:", max_tokens=1000, temperature=1.0)

In [20]:
# Tokenize starting prompt

text_generator = TextGenerator(vocab)

In [21]:
lstm.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[text_generator],
)

Epoch 1/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - loss: 1.0924
generated text:
Text: retained—possibly unseasonably our hypothesis it was , said is—or unwilling there lurking . 

1035/1035 ━━━━━━━━━━━━━━━━━━━━ 182s 172ms/step - loss: 1.0917
Epoch 2/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - loss: 0.1468
generated text:
Text: —humph grace bizarre 

1035/1035 ━━━━━━━━━━━━━━━━━━━━ 182s 176ms/step - loss: 0.1468
Epoch 3/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step - loss: 0.1351
generated text:
Text: silk lothario of the the scene , as famine by the precise 

1035/1035 ━━━━━━━━━━━━━━━━━━━━ 185s 179ms/step - loss: 0.1351
Epoch 4/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - loss: 0.1362
generated text:
Text: inaccessible the alloy quarter of order was very attracted 

1035/1035 ━━━━━━━━━━━━━━━━━━━━ 200s 177ms/step - loss: 0.1362
Epoch 5/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step - loss: 0.1225
generated text:
Text: obliterated . " now thou nothing 

1035/1035 ━━━━

## 6. Generate text using the LSTM

In [22]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        print(f"\nPROMPT: {i['prompt']}")
        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [57]:
# Prompt 1: From "The Raven"    Temperature=0.1
info = text_generator.generate(
    """Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.""", max_tokens=500, temperature=0.1
)


generated text:
Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. , and i gazed , and i have been in the 



In [58]:
print_probs(info, vocab)


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.
,:   	99.87%
.:   	0.13%
upon:   	0.0%
in:   	0.0%
to:   	0.0%
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. ,
and:   	100.0%
but:   	0.0%
i:   	0.0%
or:   	0.0%
it:   	0.0%
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volum

In [67]:
# Prompt 1: From "The Raven"    Temperature=0.5
info = text_generator.generate(
    """Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.""", max_tokens=1000, temperature=0.5
)


generated text:
Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. , and not a menial told her blended me — not that i have not been in the present , it was now the 



In [68]:
print_probs(info, vocab)


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.
,:   	75.82%
.:   	20.03%
upon:   	1.88%
in:   	0.68%
to:   	0.65%
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. ,
and:   	98.64%
but:   	0.54%
i:   	0.19%
or:   	0.15%
it:   	0.09%
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curio

In [74]:
# Prompt 1: From "The Raven"    Temperature=1.0
info = text_generator.generate(
    """Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.""", max_tokens=1000, temperature=1.0
)


generated text:
Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. imperceptibly themselves for . to de rich me — in interest and elasticity of its terrible heart will there struggled to seek the sun ebn zaiat , tended her first previous to the lips of the day . i beheld it slowly , like symptoms , while i perused them , to the opinion of fairies , such as i then my conceptions , and i have found that the nurse were no body and even three in such musical character , some far as i did it — not formerly were . — amid a word , it fell upon preparing the place of my frame , — such as my eyes — have but the dead were total 



In [75]:
print_probs(info, vocab)


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.
,:   	43.07%
.:   	22.14%
upon:   	6.78%
in:   	4.08%
to:   	4.0%
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. imperceptibly
to:   	26.89%
.:   	16.55%
,:   	9.53%
me:   	7.8%
that:   	3.17%
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint

In [108]:
# Prompt 2: From "The Tell Tale Heart"  Temperature=0.1
info = text_generator.generate(
    """And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.""", max_tokens=1000, temperature=0.1
)


generated text:
And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. , and , as i have been 



In [109]:
print_probs(info, vocab)


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.
,:   	97.74%
.:   	2.24%
—:   	0.02%
!:   	0.0%
;:   	0.0%
--------


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. ,
and:   	100.0%
or:   	0.0%
but:   	0.0%
of:   	0.0%
in:   	0.0%
--------


PROMPT: And have I not told you that what you mistake for madness is but over acuten

In [119]:
# Prompt 2: From "The Tell Tale Heart"  Temperature=0.5
info = text_generator.generate(
    """And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.""", max_tokens=1000, temperature=0.5
)


generated text:
And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. . — and the sound of her character , and i gazed upon a 



In [120]:
print_probs(info, vocab)


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.
,:   	56.83%
.:   	26.69%
—:   	10.69%
!:   	1.89%
;:   	1.25%
--------


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. .
but:   	40.59%
i:   	17.29%
in:   	11.18%
:   	9.34%
and:   	6.97%
--------


PROMPT: And have I not told you that what you mistake for madness is but over

In [129]:
# Prompt 2: From "The Tell Tale Heart"  Temperature=1.0
info = text_generator.generate(
    """And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.""", max_tokens=1000, temperature=1.0
)


generated text:
And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. ! such a medium it seemed to stone escape and upon her margin overboard the land into the case , and then atrocity glared upon the point of my efforts , it began by the first note , the person were very explanation . oh ; original air ! four ! ‘in gravitation in her employment than what was , it accurately . let us — and that , but by yours from a lack changed every 



In [130]:
print_probs(info, vocab)


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.
,:   	25.43%
.:   	17.43%
—:   	11.03%
!:   	4.64%
;:   	3.77%
--------


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. !
—:   	14.58%
but:   	7.94%
and:   	7.84%
:   	7.0%
i:   	4.18%
--------


PROMPT: And have I not told you that what you mistake for madness is but over acu

In [138]:
# Prompt 3: From the Mystery of Marie Roget   Temperature=0.1
info = text_generator.generate(
    """There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. """, max_tokens=1000, temperature=0.1
)


generated text:
There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  , and i gazed upon the 



In [139]:
print_probs(info, vocab)


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. 
,:   	100.0%
—:   	0.0%
.:   	0.0%
in:   	0.0%
and:   	0.0%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  ,
and:   	100.0%
but:   	0.0%
i:   	0.0%
it:   	0.0%
the:   	0.0%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that

In [217]:
# Prompt 3: From the Mystery of Marie Roget   Temperature=0.5
info = text_generator.generate(
    """There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. """, max_tokens=1000, temperature=0.5
)


generated text:
There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  , and i gazed , and the rich horror of her unhappy uncertain that i have been the 



In [218]:
print_probs(info, vocab)


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. 
,:   	78.5%
—:   	10.76%
.:   	4.37%
in:   	1.0%
and:   	0.85%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  ,
and:   	98.57%
but:   	0.45%
i:   	0.17%
it:   	0.17%
the:   	0.15%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a charact

In [212]:
# Prompt 3: From the Mystery of Marie Roget   Temperature=1.0
info = text_generator.generate(
    """There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. """, max_tokens=1000, temperature=1.0
)


generated text:
There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  from her interests , upon its white and obstinate or ærial steady oil — and anon , a capability of the true disease , each and incomprehensible . the coincidence design — i then be over not by a small sound , beneath it not , as to give me the idle saloon — or in the time occurred to her principles , and i fancied the springs , as mr . herein is with the immoderate eyes of a few of more nature of voice was such a palpable some time were positive , and soon like , although he slept then upon the nature of no ponder , but sufficiently 



In [213]:
print_probs(info, vocab)


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. 
,:   	28.41%
—:   	10.52%
.:   	6.71%
in:   	3.21%
and:   	2.96%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  from
the:   	24.21%
her:   	10.77%
their:   	8.21%
my:   	6.54%
a:   	5.3%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a 